In [1]:
import gurobipy as gp
import math
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter, PercentFormatter
import numpy as np
import pandas as pd
import re
import os
import shutil

In [2]:
# output generation for paper 2

In [112]:
# get input paths
test_set = "miplib_2017_5000_paper2"
instance_fldr = os.path.join("instances", test_set)
test_set_fldr = os.path.join("test_sets", test_set)
results_fldr = os.path.join("results", test_set)
out_fldr = os.path.join("outputs", test_set)

# set filters
seed_idxs = [0]  
max_indices = 100
degrees = [-1, 1]  # todo update this as needed
term_list = [4, 64]
filter_cbc = False
max_base_std = 1e10
min_termination_time = 10  # todo update this as needed - fitler wins on default runs that took at least 10 seconds
short, medium, long = 60, 600, 3600
remove_status_changes = False
win_threshold = .2
filter_redundant = True

default_generators = ["None", "New", "Farkas"]
# need to redownload the test results for the others
test_generators = ["NoDisjunction"]  # "All", "Disjunction", "NoDisjunction" ,"Matrix", "Term", "Basis", "NoMatrix", "NoTerm", "NoBasis"]
generators = default_generators + test_generators
disjunctive_generators = [g for g in generators if g != "None"]
parametric_generators = [g for g in generators if g not in ["None", "New"]]


# set up some mappings
cat_map_new_lines = {
    "None": "Default",
    "Farkas": "Param Disj,\nParam Cuts",
    "Old": "Param Disj,\nCalc Cuts",
    "New": "Calc Disj,\nCalc Cuts",
    "All": "Prune and\nSupport",
    "Disjunction": "Prune\nDisjunction",
    "Matrix": "Support\nMatrix",
    "Term": "Support\nTerm",
    "Basis": "Support\nBasis",
    "NoDisjunction": "Support"
}
cat_map = {
    "None": "Default",
    "Farkas": "Param Disj, Param Cuts",
    "Old": "Param Disj, Calc Cuts",
    "New": "Calc Disj, Calc Cuts",
    "All": "Prune and Support",
    "Disjunction": "Prune Disjunction",
    "Matrix": "Strengthen Matrix",
    "Term": "Strengthen Term",
    "Basis": "Strengthen Basis",
    "NoDisjunction": "Support"
}
perturbation_map = {
    "matrix": "Coefficient Matrix",
    "rhs": "Right Hand Side",
    "objective": "Objective"
}
label = {
    "postRootTime": "Time after Processing Root nodes",
    "rootDualBoundTimeSansVpc": "Root Processing Time (Minus VPC Generation)",
    "terminationTimeSansVpc": "Time (Minus VPC Generation)",
    "terminationTime": "Time",
    "nodes": "Nodes Processed",
    "iterations": "LP iterations",
}
unit = {
    "postRootTime": "(seconds)",
    "rootDualBoundTimeSansVpc": "(seconds)",
    "terminationTimeSansVpc": "(seconds)",
    "terminationTime": "(seconds)",
    "nodes": "(1000 nodes)",
    "iterations": "(1000 iterations)",
}
limits = {
    "postRootTime": 7200,
    "terminationTimeSansVpc": 7200,
    "terminationTime": 7200,
    "rootDualBoundTimeSansVpc": 5,
    "nodes": 10000,
    "iterations": 37500
}
bracket_bounds = {
    "short": (min_termination_time, short),
    "medium": (short, medium),
    "long": (medium, long)
}
param_map = {
    "degree": "Degree of Perturbation",
    "terms": "Number of Disjunctive Terms",
}

In [4]:
# matplotlib settings
plt.rc('text', usetex=True)  # use latex fonts
plt.rcParams['font.size'] = 18
plt.rcParams['figure.titlesize'] = 24
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 14

## Check run failures

In [5]:
# check if each folder in test_set_fldr has a corresponding .mps file in instance_fldr
# for instance in os.listdir(test_set_fldr):
#     if not os.path.isdir(os.path.join(test_set_fldr, instance)):
#         continue
#     if not os.path.exists(os.path.join(instance_fldr, f"{instance}.mps")):
#         # remove the folder if the instance is missing
#         # shutil.rmtree(os.path.join(test_set_fldr, instance))
#         print(f"Removed {instance} from test set")

In [6]:
# running list of strings contained by different error codes
# last two are catchalls
err = {
    "walltime": [],
    "bad_alloc": [],
    "out of memory": [],
    "vmem": [],
    "takeoffcuts": [],
    "solver is dual infeasible": [],
    "solver must be optimal": [],
    "segmentation fault": [],
    "no vpcs were made from a new disjunction": [],
    "must have primalbound >= root lp objective": [],
    "objective at parent nodes": [],
    "failed to optimize mip": [],
    "disjunction does not represent a full binary tree": [],
    "solver not proven optimal for nodes": [],
    "unable to open": [],
    "license": [],
    "dot product with obj differs from solver": [],
    "gurobi: error during callback: addCut": [],
    "cglvpc::setupconstraints: objective at disjunctive term": [],
    "unable to read file": [],
    "stats.id == stats_vec": [],
    "size of our disjunction is not what we expected it to be": [],
    "dimension must stay fixed": [],
    "vpcgenerator must be": [],
    "objective values must match": [],
    "objective at disjunctive term": [],
}

# read in cbc acceptable instances from cbc.txt
with open("cbc.txt", "r") as f:
    cbc_instances = f.read().split("\n")

# runs that errored out with new error code
other = []

# runs that had no errors
empty = []

# runs that only had warnings
warn_strs = ["warning", "prlp is primal infeasible", "farkas", "x:", "x[", "b:",
             "b[", "v:", "v[", "cut:", "A_i . x", "dot product with obj differs from solver"]
warning = []

# series that didn't run
no_go = []

# track sizes of instances
rows, cols, density = {}, {}, {}

# map the names
names = {}

# counts
count_series = 0
count_instances = 0
number_instances = {}

# iterate over all expected runs
for instance in os.listdir(test_set_fldr):
    if not os.path.isdir(os.path.join(test_set_fldr, instance)):
        continue
    # only look at cbc instances if we ran with cbc
    if instance not in cbc_instances and "gurobi" not in test_set and filter_cbc:
        continue
        
    # get the number of rows and columns in the instance
    mdl = gp.read(os.path.join(instance_fldr, f"{instance}.mps"))
    rows[instance] = mdl.NumConstrs
    cols[instance] = mdl.NumVars
    density[instance] = mdl.NumNZs / (mdl.NumConstrs * mdl.NumVars)
        
    for perturbation in os.listdir(os.path.join(test_set_fldr, instance)):
        if not os.path.isdir(os.path.join(test_set_fldr, instance, perturbation)):
            continue
        # only look at perturbations that were run
        p, d = perturbation.split("_")
        if int(d) not in degrees or p not in perturbation_map:
            continue
        for terms in term_list:
            for generator in generators:
                for seed_idx in seed_idxs:

                    # set variables for this iterations
                    count_series += 1
                    stem = f"{instance}_{perturbation}_{terms}_{generator}_{seed_idx}"
                    file_pth = os.path.join(results_fldr, f"{stem}.err")
                    series_fldr = os.path.join(test_set_fldr, instance, perturbation)
                    current_count = len([f for f in os.listdir(series_fldr) if f.endswith(".mps")])
                    count_instances += current_count
                    names[stem] = instance
                    number_instances[stem] = {
                        "expected": current_count,
                        "recorded": 0,
                        "generator": generator,
                        "error": "N/A"
                    }
    
                    # check if the series wasn't run
                    if not os.path.exists(file_pth):
                        number_instances[stem]["error"] = "no go"
                        no_go.append(stem)
                    
                    # check if the series ran with no errors or warnings
                    elif os.path.getsize(file_pth) == 0:
                        number_instances[stem]["error"] = "empty"
                        empty.append(stem)
                    
                    # track which error codes were thrown
                    else:
                        # read the file
                        with open(file_pth, "r") as f:
                            text = f.read().lower()
                        
                        # assign the error file to the appropriate list
                        found_code = False
                        for code in err:
                            if code in text:
                                if code == "dot product with obj differs from solver":
                                    pattern = r"obj viol from solver: (-?\d+\.\d+)\. calculated: (-?\d+\.\d+)"
                                    s, c = re.findall(pattern, text)[-1]
                                    # if we didn't terminate, this isn't an error, so keep going
                                    if abs(float(s) - float(c)) < 1e-3:
                                        continue
                                err[code].append(stem)
                                found_code = True
                                number_instances[stem]["error"] = code
                                break
                        if not found_code:
                            if all(not line or any(w in line for w in warn_strs) for line in text.splitlines()):
                                warning.append(stem)
                                number_instances[stem]["error"] = "warning"
                            else:
                                other.append(stem)
                                number_instances[stem]["error"] = "other"

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Read MPS format model from file instances/miplib_2017_5000_paper2/bienst2.mps
Reading time = 0.00 seconds
bienst2: 576 rows, 505 columns, 2184 nonzeros
Read MPS format model from file instances/miplib_2017_5000_paper2/set3-15.mps
Reading time = 0.00 seconds
set3-15: 3747 rows, 4019 columns, 13747 nonzeros
Read MPS format model from file instances/miplib_2017_5000_paper2/f2gap801600.mps
Reading time = 0.00 seconds
f2gap801600: 80 rows, 1600 columns, 3200 nonzeros
Read MPS format model from file instances/miplib_2017_5000_paper2/stein15inf.mps
Reading time = 0.00 seconds
stein15inf: 37 rows, 15 columns, 135 nonzeros
Read MPS format model from file instances/miplib_2017_5000_paper2/neos-3610173-itata.mps
Reading time = 0.00 seconds
neos-3610173-itata: 747 rows, 844 columns, 2130 nonzeros
Read MPS format model from file instances/miplib_2017_5000_paper2/10teams.mps
Reading time = 0.00 seconds
10TEAMS

In [7]:
# check which series didn't run
print(no_go)

[]


In [8]:
# get the proportion of series that at least got started
1 - (len(no_go) / count_series)

1.0

In [9]:
# out of time - got hung up in code somewhere - ok
print(err["walltime"])
len(err["walltime"]) / count_series

['cod105_rhs_1_64_New_0', 'cod105_rhs_1_64_Farkas_0', 'cod105_rhs_1_64_NoDisjunction_0', 'cod105_objective_1_64_New_0', 'cod105_objective_1_64_Farkas_0', 'cod105_matrix_1_64_New_0', 'cod105_matrix_1_64_Farkas_0', 'cod105_objective_-1_64_New_0', 'neos-1605061_rhs_1_64_New_0']


0.0009648370497427101

In [10]:
# out of memory - memory is maxed already - this is what it is
# todo: figure out where we ran short on memory so we can explain why we dropped them
print(err["bad_alloc"] + err["out of memory"] + err["vmem"])
len(err["bad_alloc"] + err["out of memory"] + err["vmem"]) / count_series

['f2gap801600_objective_1_64_New_0', '10teams_objective_-1_64_New_0', 'piperout-d27_objective_1_64_New_0', 'piperout-d27_objective_1_64_Farkas_0', 'piperout-d27_objective_1_64_NoDisjunction_0', 'piperout-d27_objective_-1_64_New_0', 'piperout-d27_objective_-1_64_Farkas_0', 'piperout-d27_objective_-1_64_NoDisjunction_0', 'piperout-d20_objective_1_64_New_0', 'piperout-d20_objective_1_64_NoDisjunction_0', 'piperout-d20_objective_-1_64_New_0', 'piperout-d20_objective_-1_64_Farkas_0', 'nexp-150-20-1-5_matrix_1_64_NoDisjunction_0', 'nexp-150-20-1-5_rhs_-1_64_NoDisjunction_0', 'qnet1_matrix_1_64_New_0', 'neos-2328163-agri_objective_1_64_New_0', 'neos-2328163-agri_matrix_1_64_New_0', 'neos-2328163-agri_objective_-1_64_New_0', 'mod010_objective_1_64_New_0', 'mod010_objective_1_64_Farkas_0', 'mod010_matrix_1_64_New_0', 'mod010_matrix_-1_64_New_0', 'mod010_matrix_-1_64_NoDisjunction_0', 'mod010_objective_-1_64_New_0', 'mod010_objective_-1_64_Farkas_0', 'app2-2_objective_1_64_New_0', 'app2-2_object

0.025621783876500857

In [11]:
# rerun this if want to give more memory to some instances
# bad_alloc_names = set(n.split("_")[0] for n in err["bad_alloc"])
# mem = pd.read_csv("more_memory.csv", index_col=0)
# mem["reason"] = "hard solve" 
# 
# for n in bad_alloc_names:
#     if f"{n}.mps" not in mem.index:
#         new_row = pd.DataFrame([{'file_name': f"{n}.mps", 'memory': 16.0, 'reason': 'big disjunction'}]).set_index('file_name')
#         mem = pd.concat([mem, new_row])
#     else:
#         mem.loc[f'{n}.mps', 'memory'] = 16.0
# 
# mem.to_csv("more_memory.csv")

In [12]:
# this is an issue with John's bookkeeping - not much we can do here
print(err["takeoffcuts"])
len(err["takeoffcuts"]) / count_series

[]


0.0

In [13]:
print(err["solver is dual infeasible"])
len(err["solver is dual infeasible"]) / count_series

[]


0.0

In [14]:
# these are usually issues with CLP finding optimality - not much we can do here
print(err["solver must be optimal"])
len(err["solver must be optimal"]) / count_series

[]


0.0

In [15]:
print(err["segmentation fault"])
len(err["segmentation fault"]) / count_series

['neos-3665875-lesum_rhs_1_64_New_0']


0.0001072041166380789

In [16]:
# seg_err = {
#     "Bad image at line": [],
# }
# 
# seg_other = []
# 
# for stem in err["segmentation fault"]:
#     file_pth = os.path.join(results_fldr, f"{stem}.out")
# 
#     with open(file_pth, "r") as f:
#         text = f.read()
#     
#     # assign the error file to the appropriate list
#     found_code = False
#     for code in seg_err:
#         if code in text:
#             seg_err[code].append(stem)
#             found_code = True
#             break
#     if not found_code:
#         seg_other.append(stem)

In [17]:
# print(seg_err["Bad image at line"])
# len(seg_err["Bad image at line"]) / len(err["segmentation fault"]) if err["segmentation fault"] else 0

In [18]:
# print(seg_other)
# len(seg_other)/len(err["segmentation fault"]) if err["segmentation fault"] else 0

In [19]:
# # get breakdown of why vpc generation failed - mostly from lack of provisioning
# for code, exps in seg_err.items():
#     print(f"{code}: {len(exps) / len(err['segmentation fault']) if err['segmentation fault'] else 0}")
# 
# print(f"other: {len(seg_other) / len(err['segmentation fault']) if err['segmentation fault'] else 0}")

In [20]:
# todo: check aleks' removals and drop those below for similar reasons
# todo: check size of disjunctions and decide what to do with those that are too big
# these should all be from the problem being too big and hitting the time limit or integer solutions
print(err["no vpcs were made from a new disjunction"])
missing_4_term = [n for n in err["no vpcs were made from a new disjunction"] if "_4_" in n]
missing_64_term = [n for n in err["no vpcs were made from a new disjunction"] if "_64_" in n]
print(f'4 term: {len(missing_4_term) / count_series}')
print(f'64 term: {len(missing_64_term) / count_series}')

['bienst2_rhs_1_64_New_0', 'bienst2_rhs_1_64_Farkas_0', 'bienst2_rhs_1_64_NoDisjunction_0', 'bienst2_objective_1_64_New_0', 'bienst2_objective_1_64_Farkas_0', 'bienst2_matrix_-1_64_New_0', 'bienst2_matrix_-1_64_Farkas_0', 'bienst2_matrix_-1_64_NoDisjunction_0', 'bienst2_objective_-1_64_New_0', 'bienst2_objective_-1_64_Farkas_0', 'bienst2_objective_-1_64_NoDisjunction_0', 'neos-555343_rhs_1_4_New_0', 'neos-555343_rhs_1_4_Farkas_0', 'neos-555343_rhs_1_4_NoDisjunction_0', 'neos-555343_rhs_1_64_New_0', 'neos-555343_rhs_1_64_Farkas_0', 'neos-555343_rhs_1_64_NoDisjunction_0', 'neos-555343_objective_1_4_New_0', 'neos-555343_objective_1_4_Farkas_0', 'neos-555343_objective_1_4_NoDisjunction_0', 'neos-555343_objective_1_64_New_0', 'neos-555343_objective_1_64_Farkas_0', 'neos-555343_objective_1_64_NoDisjunction_0', 'neos-555343_matrix_1_4_New_0', 'neos-555343_matrix_1_4_Farkas_0', 'neos-555343_matrix_1_4_NoDisjunction_0', 'neos-555343_matrix_1_64_New_0', 'neos-555343_matrix_1_64_Farkas_0', 'neos-

In [21]:
# vpc_err = {
#     "CglVPC: Finishing with exit reason: PRLP_TIME_LIMIT": [],
#     "CglVPC: Finishing with exit reason: TIME_LIMIT": [],
#     "CglVPC: Finishing with exit reason: NO_CUTS_LIKELY": [],
#     "CglVPC: Finishing with exit reason: PRLP_INFEASIBLE": [],
#     "CglVPC: Finishing with exit reason: SUCCESS": [],
#     "CglVPC: Finishing with exit reason: OPTIMAL_SOLUTION_FOUND": [],
#     "CglVPC: Finishing with exit reason: FAIL_LIMIT": [],
#     "CglVPC: Finishing with exit reason: NO_DISJUNCTION": [],
# }
# 
# vpc_other = []
# 
# for stem in err["no vpcs were made from a new disjunction"]:
#     file_pth = os.path.join(results_fldr, f"{stem}.out")
# 
#     with open(file_pth, "r") as f:
#         text = f.read()
#     
#     # assign the error file to the appropriate list
#     found_code = False
#     for code in vpc_err:
#         if code in text:
#             vpc_err[code].append(stem)
#             found_code = True
#             break
#     if not found_code:
#         vpc_other.append(stem)

In [22]:
# print(vpc_err["CglVPC: Finishing with exit reason: PRLP_TIME_LIMIT"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: PRLP_TIME_LIMIT"]) / len(err["no vpcs were made from a new disjunction"])

In [23]:
# print(vpc_err["CglVPC: Finishing with exit reason: TIME_LIMIT"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: TIME_LIMIT"]) / len(err["no vpcs were made from a new disjunction"])

In [24]:
# print(vpc_err["CglVPC: Finishing with exit reason: NO_CUTS_LIKELY"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: NO_CUTS_LIKELY"]) / len(err["no vpcs were made from a new disjunction"])

In [25]:
# print(vpc_err["CglVPC: Finishing with exit reason: PRLP_INFEASIBLE"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: PRLP_INFEASIBLE"]) / len(err["no vpcs were made from a new disjunction"])

In [26]:
# print(vpc_err["CglVPC: Finishing with exit reason: SUCCESS"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: SUCCESS"]) / len(err["no vpcs were made from a new disjunction"])

In [27]:
# print(vpc_err["CglVPC: Finishing with exit reason: OPTIMAL_SOLUTION_FOUND"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: OPTIMAL_SOLUTION_FOUND"]) / len(err["no vpcs were made from a new disjunction"])

In [28]:
# print(vpc_err["CglVPC: Finishing with exit reason: FAIL_LIMIT"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: FAIL_LIMIT"]) / len(err["no vpcs were made from a new disjunction"])

In [29]:
# print(vpc_err["CglVPC: Finishing with exit reason: NO_DISJUNCTION"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: NO_DISJUNCTION"]) / len(err["no vpcs were made from a new disjunction"])

In [30]:
# vpc_other

In [31]:
# # get breakdown of why vpc generation failed - mostly from lack of provisioning/problem being too large
# if err["no vpcs were made from a new disjunction"]:
#     for code, exps in vpc_err.items():
#         print(f"{code}: {len(exps) / len(err['no vpcs were made from a new disjunction'])}")
#     
#     print(f"other: {len(vpc_other) / len(err['no vpcs were made from a new disjunction'])}")

In [32]:
print(err["must have primalbound >= root lp objective"])
len(err["must have primalbound >= root lp objective"]) / count_series

['neos4_objective_1_4_None_0', 'neos4_objective_1_64_None_0']


0.0002144082332761578

In [33]:
# LP relaxation objective is not going to match root nodes objective when warm starting 
print(err["objective at parent nodes"])
len(err["objective at parent nodes"]) / count_series

[]


0.0

In [34]:
# not enough tolerance added to bound (or we hit time limit) - element 2 from 5 and 4 from 4
print(err["failed to optimize mip"])
len(err["failed to optimize mip"]) / count_series

[]


0.0

In [35]:
# todo: figure out why
print(err["disjunction does not represent a full binary tree"])
len(err["disjunction does not represent a full binary tree"]) / count_series

[]


0.0

In [36]:
# again issue with not getting through vpc generation in time
# todo: handle this gracefully
print(err["solver not proven optimal for nodes"])
len(err["solver not proven optimal for nodes"]) / count_series

[]


0.0

In [37]:
print(err["unable to open"])
len(err["unable to open"]) / count_series

[]


0.0

In [38]:
print(err["license"])
len(err["license"]) / count_series

[]


0.0

In [39]:
print(warning)
len(warning) / count_series

['bienst2_rhs_1_4_New_0', 'bienst2_rhs_1_4_Farkas_0', 'bienst2_objective_1_4_New_0', 'bienst2_objective_1_4_Farkas_0', 'bienst2_objective_1_4_NoDisjunction_0', 'bienst2_matrix_-1_4_New_0', 'bienst2_matrix_-1_4_NoDisjunction_0', 'bienst2_objective_-1_4_New_0', 'bienst2_objective_-1_4_Farkas_0', 'bienst2_objective_-1_4_NoDisjunction_0', '10teams_objective_1_4_New_0', '10teams_matrix_1_64_New_0', '10teams_matrix_1_64_Farkas_0', '10teams_objective_-1_4_New_0', 'gmu-35-40_objective_1_4_New_0', 'gmu-35-40_objective_1_4_Farkas_0', 'gmu-35-40_objective_1_4_NoDisjunction_0', 'gmu-35-40_objective_1_64_New_0', 'gmu-35-40_matrix_-1_4_New_0', 'gmu-35-40_matrix_-1_4_Farkas_0', 'gmu-35-40_matrix_-1_4_NoDisjunction_0', 'gmu-35-40_objective_-1_4_New_0', 'gmu-35-40_objective_-1_4_Farkas_0', 'gmu-35-40_objective_-1_4_NoDisjunction_0', 'gmu-35-40_objective_-1_64_New_0', 'neos-3610051-istra_rhs_-1_64_New_0', 'neos-3610051-istra_matrix_-1_4_New_0', 'neos-3610051-istra_objective_-1_64_New_0', 'ci-s4_objectiv

0.056067753001715265

In [40]:
# errors unaccounted for
print(other)
len(other) / count_series

['neos-3083819-nubu_matrix_-1_4_New_0', 'neos-3083819-nubu_matrix_-1_64_New_0', 'eil33-2_objective_1_4_New_0']


0.00032161234991423673

In [41]:
# proportion of series that were improperly provisioned
(len(err["bad_alloc"] + err["out of memory"] + err["walltime"] + err["vmem"])) / count_series

0.026586620926243566

In [42]:
# todo handle this
print(err["dot product with obj differs from solver"])
len(err["dot product with obj differs from solver"]) / count_series

['neos-3592146-hawea_matrix_-1_4_New_0']


0.0001072041166380789

In [43]:
# changed code to ignore this error
print(err["gurobi: error during callback: addCut"])
len(err["gurobi: error during callback: addCut"]) / count_series

[]


0.0

In [44]:
# largely not replicating - only issue I could find was aleks missing updated objective from CLP when resolving to check this
print(err["cglvpc::setupconstraints: objective at disjunctive term"])
len(err["cglvpc::setupconstraints: objective at disjunctive term"]) / count_series

[]


0.0

In [45]:
# not replicating - rerun
print(err["unable to read file"])
len(err["unable to read file"]) / count_series

[]


0.0

In [46]:
# not replicating - rerun
print(err["stats.id == stats_vec"])
len(err["stats.id == stats_vec"]) / count_series

[]


0.0

In [47]:
print(err["size of our disjunction is not what we expected it to be"])
len(err["size of our disjunction is not what we expected it to be"]) / count_series

[]


0.0

In [48]:
print(err["vpcgenerator must be"])
len(err["vpcgenerator must be"]) / count_series

[]


0.0

In [49]:
print(err["dimension must stay fixed"])
len(err["dimension must stay fixed"]) / count_series

[]


0.0

In [50]:
print(err["objective values must match"])
len(err["objective values must match"]) / count_series

['f2gap801600_objective_-1_64_New_0', 'neos-3610173-itata_matrix_-1_4_New_0', 'neos-3610051-istra_matrix_-1_64_New_0', 'f2gap401600_objective_1_64_New_0', 'traininstance6_objective_-1_4_New_0', 'traininstance6_objective_-1_64_New_0', 'mas74_matrix_1_4_New_0', 'mas74_matrix_1_64_New_0', 'mas74_matrix_-1_64_New_0', 'rentacar_objective_-1_4_New_0', 'neos-3421095-cinca_objective_1_64_New_0', 'irp_objective_-1_4_New_0', 'aligninq_matrix_1_4_New_0', 'neos-631517_matrix_-1_64_New_0', 'neos-3610040-iskar_matrix_-1_64_New_0', 'neos-3627168-kasai_matrix_-1_64_New_0', 'pg_rhs_1_64_New_0', 'neos-3611689-kaihu_matrix_-1_64_New_0', 'neos-3611689-kaihu_objective_-1_64_New_0', 'mas76_matrix_1_64_New_0', 'mas76_matrix_-1_64_New_0', 'neos-3754480-nidda_objective_1_4_New_0', 'neos-3754480-nidda_objective_1_64_New_0', 'neos-3754480-nidda_rhs_-1_4_New_0', 'neos-3754480-nidda_rhs_-1_64_New_0', 'neos-3754480-nidda_objective_-1_64_New_0', 'control30-3-2-3_matrix_-1_64_New_0', 'control30-3-2-3_objective_-1_64_

0.0037521440823327615

In [51]:
print(err["objective at disjunctive term"])
len(err["objective at disjunctive term"]) / count_series

['neos-631517_matrix_1_4_New_0', 'neos-631517_matrix_1_64_New_0', 'gus-sch_matrix_1_4_New_0', 'gus-sch_matrix_1_64_New_0', 'neos-5182409-nasivi_matrix_-1_4_New_0', 'neos-5182409-nasivi_matrix_-1_64_New_0', 'roll3000_matrix_-1_4_New_0', 'roll3000_matrix_-1_64_New_0', 'control30-3-2-3_matrix_-1_4_New_0']


0.0009648370497427101

In [52]:
# get breakdown of errors
for code, exps in err.items():
    print(f"{code}: {len(exps) / count_series}")

print(f"other: {len(other) / count_series}")

print(f"warning: {len(warning) / count_series}")

print(f"no errors/warnings: {len(empty) / count_series}")

print(f"no go: {len(no_go) / count_series}")

walltime: 0.0009648370497427101
bad_alloc: 0.021655231560891938
out of memory: 0.0011792452830188679
vmem: 0.0027873070325900515
takeoffcuts: 0.0
solver is dual infeasible: 0.0
solver must be optimal: 0.0
segmentation fault: 0.0001072041166380789
no vpcs were made from a new disjunction: 0.2928816466552316
must have primalbound >= root lp objective: 0.0002144082332761578
objective at parent nodes: 0.0
failed to optimize mip: 0.0
disjunction does not represent a full binary tree: 0.0
solver not proven optimal for nodes: 0.0
unable to open: 0.0
license: 0.0
dot product with obj differs from solver: 0.0001072041166380789
gurobi: error during callback: addCut: 0.0
cglvpc::setupconstraints: objective at disjunctive term: 0.0
unable to read file: 0.0
stats.id == stats_vec: 0.0
size of our disjunction is not what we expected it to be: 0.0
dimension must stay fixed: 0.0
vpcgenerator must be: 0.0
objective values must match: 0.0037521440823327615
objective at disjunctive term: 0.000964837049742

## Read in data

In [53]:
# map generator names to the corresponding data frames
df_map = {g: pd.DataFrame() for g in generators} 
gap_map = {g: pd.DataFrame() for g in generators}
regex = re.compile(r'([a-zA-Z0-9-]+(?:_o)?)_([a-z]+)_([0-9-]+)_([0-9]+)_([a-zA-Z ]+)')
solution_pattern = r"_(\d+)\.pb"

# declaring types as needed
column_types = {
    "lpBound": float,
    "lpBoundPostVpc": float,
    "disjunctiveDualBound": float,
    "primalBound": float,
    "rootDualBound": float,
    "dualBound": float
}

skipped_instances = set()
primal_bounds = {}
same_solution = {}

# iterate over all files in the folder
for file_name in os.listdir(results_fldr):
    
    file_pth = os.path.join(results_fldr, file_name)
    
    # if the file is not a nonempty csv, skip it
    if not file_name.endswith(".csv") or os.path.getsize(file_pth) == 0:
        continue
    
    # get the experimental set up
    match = regex.search(file_name)
    instance_name = names.get(file_name[:-4])
    if not instance_name:
        skipped_instances.add(file_name[:-4].split("_")[0])
        os.remove(file_pth)
        continue
    # instance_name = match.group(1)
    perturbation = match.group(2)
    assert perturbation in ["matrix", "rhs", "bound", "objective"], f"Unknown perturbation: {perturbation}"
    expo = int(match.group(3))
    assert expo in degrees, f"Unknown degree: {expo}"
    degree = 2**int(expo)
    terms = int(match.group(4))
    assert terms in term_list, f"Unknown number of terms: {terms}"
    generator = match.group(5)
    assert generator in generators, f"Unknown generator: {generator}"
    base_name = f"{instance_name}_0"
    
    # get the primal bounds for this experiment
    cur_instance_test_set_fldr = os.path.join(test_set_fldr, instance_name, f"{perturbation}_{expo}")
    for test_set_file in os.listdir(cur_instance_test_set_fldr):
        if test_set_file.endswith(".pb"):
            with open(os.path.join(cur_instance_test_set_fldr, test_set_file), "r") as f:
                primal_bounds[perturbation, expo, ".".join(test_set_file.split(".")[:-1])] = float(f.read())
                
    # see if solution changed
    for test_set_file in os.listdir(cur_instance_test_set_fldr):
        if test_set_file.endswith(".pb"):
            perturbation_name = ".".join(test_set_file.split(".")[:-1])
            same_solution[perturbation, expo, perturbation_name] = \
                primal_bounds[perturbation, expo, base_name] == primal_bounds[perturbation, expo, perturbation_name]
            
    # read the file
    df = pd.read_csv(file_pth, keep_default_na=False, dtype=column_types, index_col=0)
    
    for instance_idx in df.index:
        
        # fill in primal bounds if missing
        # df.loc[instance_idx, "primalBound"] = min(primal_bounds.get(stem_map.get(instance_idx), 1e100), df.loc[instance_idx, "primalBound"])
        df.loc[instance_idx, "primalBound"] = min(
            primal_bounds[perturbation, expo, f"{instance_name}_{instance_idx}"], df.loc[instance_idx, "primalBound"]
        )
        
        # same with root dual bound
        df.loc[instance_idx, "rootDualBound"] = df.loc[instance_idx, "rootDualBound"] if df.loc[instance_idx, "rootDualBound"] < 1e100 else df.loc[instance_idx, "lpBoundPostVpc"] 
    
    # get rid of the index so the rest of the notebook works
    df.reset_index(inplace=True)
    
    # add some identifying columns
    df["instance"] = instance_name
    df["perturbation"] = perturbation
    df["degree"] = degree
    df["terms"] = terms
    df["rows"] = rows[instance_name]
    df["cols"] = cols[instance_name]
    df["density"] = density[instance_name]
    
    # append to the appropriate data frame
    df_map[generator] = pd.concat([df_map[generator], df])
    
    # track recorded vs expected experiments
    number_instances[file_name[:-4]]["recorded"] = len(df)

In [54]:
# convert number_instances to dataframe
frame = pd.DataFrame(number_instances).T
frame.head()

,expected,recorded,generator,error
bienst2_rhs_1_4_None_0,6,6,None,empty
bienst2_rhs_1_4_New_0,6,6,New,warning
bienst2_rhs_1_4_Farkas_0,6,6,Farkas,warning
bienst2_rhs_1_4_NoDisjunction_0,6,6,NoDisjunction,empty
bienst2_rhs_1_64_None_0,6,6,None,empty


In [55]:
# redo the runs that have incomplete data that we're not sure should be that way
redos = frame.loc[(frame["expected"] > frame["recorded"]) & (frame["error"] != "no vpcs were made from a new disjunction")].index.tolist()
redos = pd.DataFrame({"experiment": redos})
redos.to_csv("redos.csv", index=False)

In [56]:
if "miplib" in test_set or "quick" in test_set:
    # group frame by generator and sum remaining columns
    gb = frame.groupby(["generator", "error"]).sum().reset_index()
    gb["missing"] = gb["expected"] - gb["recorded"]
    total = gb.groupby("generator")[["expected", "missing"]].sum().reset_index()
    gb = pd.merge(gb, total, on="generator", suffixes=("", " total"))
    gb["ratio missing (by generator)"] = gb["missing"] / gb["missing total"]
    gb["ratio missing (by generator)"] = gb["ratio missing (by generator)"].apply(lambda x: round(x, 4))
    gb = gb.loc[:, ~gb.columns.str.contains("total")]  # get rid of the total columns
    gb.set_index(["generator", "error"], inplace=True)
    gb.to_csv(os.path.join(out_fldr, "missing_table.csv"), index=False, mode="w")
else:
    gb = None
gb

expected recorded  \
generator     error                                                          
Farkas        bad_alloc                                       284       95   
              empty                                          6419     6419   
              no vpcs were made from a new disjunction       4597        0   
              out of memory                                     5        4   
              vmem                                             63       21   
              walltime                                         10        0   
              warning                                         720      720   
New           bad_alloc                                       788      335   
              dot product with obj differs from solver          4        2   
              empty                                          4978     4870   
              no vpcs were made from a new disjunction       4586        0   
              objective at disjunctive term                    41       19   
              objective values must match                     203      133   
              other                                            18       14   
              out of memory                                    39       22   
              segmentation fault                                5        0   
              vmem                                             46       11   
              walltime                                         19        0   
              warning                                        1371     1199   
NoDisjunction bad_alloc                                       258       75   
              empty                                          6397     6397   
              no vpcs were made from a new disjunction       4638        0   
              out of memory                                     5        4   
              vmem                                             44       10   
              walltime                                          4        0   
              warning                                         752      752   
None          empty                                         12053    12038   
              must have primalbound >= root lp objective       12        8   
              out of memory                                    10        4   
              vmem                                             23        8   

                                                         missing  \
generator     error                                                
Farkas        bad_alloc                                      189   
              empty                                            0   
              no vpcs were made from a new disjunction      4597   
              out of memory                                    1   
              vmem                                            42   
              walltime                                        10   
              warning                                          0   
New           bad_alloc                                      453   
              dot product with obj differs from solver         2   
              empty                                          108   
              no vpcs were made from a new disjunction      4586   
              objective at disjunctive term                   22   
              objective values must match                     70   
              other                                            4   
              out of memory                                   17   
              segmentation fault                               5   
              vmem                                            35   
              walltime                                        19   
              warning                                        172   
NoDisjunction bad_alloc                                      183   
              empty                                            0

In [57]:
for gen in generators:
    print(gen)
    masks = {
        0: -1e20 > df_map[gen]["lpBound"],
        1: df_map[gen]["lpBound"] - 1e-3 > df_map[gen]["lpBoundPostVpc"],
        2: (df_map[gen]["lpBoundPostVpc"] - 1e-3 > df_map[gen]["disjunctiveDualBound"]) & ((gen == "None") | (gen == "New")),
        3: df_map[gen]["rootDualBound"] - 1e-3 > df_map[gen]["dualBound"],
        4: (df_map[gen]["dualBound"] - 1e-3 > df_map[gen]["primalBound"]) & (df_map[gen]["dualBound"] / df_map[gen]["primalBound"] > 1 + 1e-3),
        5: df_map[gen]["primalBound"] > 1e20,
        6: 0 > df_map[gen]["vpcGenerationTime"],
        7: df_map[gen]["vpcGenerationTime"] - 1e-3 > df_map[gen]["rootDualBoundTime"],
        8: df_map[gen]["rootDualBoundTime"] - 1e-3 > df_map[gen]["terminationTime"],
        9: df_map[gen]["vpcGenerationTime"] - 1e-3 > df_map[gen]["bestSolutionTime"],
        10: df_map[gen]["bestSolutionTime"] - 1e-3 > df_map[gen]["terminationTime"]
    }
    for i, mask in masks.items():
        print(f"{gen} {i}: {mask.sum() / len(df_map[gen])}")

None
None 0: 0.0
None 1: 0.0
None 2: 0.0
None 3: 0.0
None 4: 0.0006634599436059048
None 5: 0.0
None 6: 0.0
None 7: 0.0
None 8: 0.0
None 9: 0.0
None 10: 0.0
New
New 0: 0.0
New 1: 0.0
New 2: 0.0
New 3: 0.0
New 4: 0.0009084027252081756
New 5: 0.0
New 6: 0.0
New 7: 0.0
New 8: 0.0
New 9: 0.0
New 10: 0.0
Farkas
Farkas 0: 0.0
Farkas 1: 0.0
Farkas 2: 0.0
Farkas 3: 0.0
Farkas 4: 0.0008265601322496211
Farkas 5: 0.0
Farkas 6: 0.0
Farkas 7: 0.0
Farkas 8: 0.0
Farkas 9: 0.0
Farkas 10: 0.0
NoDisjunction
NoDisjunction 0: 0.0
NoDisjunction 1: 0.0
NoDisjunction 2: 0.0
NoDisjunction 3: 0.0
NoDisjunction 4: 0.0009671179883945841
NoDisjunction 5: 0.0
NoDisjunction 6: 0.0
NoDisjunction 7: 0.0
NoDisjunction 8: 0.0
NoDisjunction 9: 0.0
NoDisjunction 10: 0.0


In [58]:
for gen, df in df_map.items():
    print(f"{gen}: {df.size}")

None: 458204
New: 250990
Farkas: 275842
NoDisjunction: 275044


In [59]:
# it shouldn't be possible that dual bound > primal bound. this only happens when we use the saved primal bound, which was used to set the dual bound
df_map["Farkas"][masks[0]]

/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_46591/3095696856.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_map["Farkas"][masks[0]]


,instanceIndex,seedIndex,vpcGenerator,terms,lpBound,disjunctiveDualBound,lpBoundPostVpc,rootDualBound,dualBound,primalBound,...,tighten_disjunction,tighten_matrix_perturbation,tighten_infeasible_to_feasible_term,tighten_feasible_to_infeasible_basis,instance,perturbation,degree,rows,cols,density


In [60]:
for gen in df_map:
    mask = (-1e20 > df_map[gen]["lpBound"]) | \
        (df_map[gen]["lpBound"] - 1e-3 > df_map[gen]["lpBoundPostVpc"]) | \
        ((df_map[gen]["lpBoundPostVpc"] - 1e-3 > df_map[gen]["disjunctiveDualBound"]) & (gen != "Farkas")) | \
        (df_map[gen]["rootDualBound"] - 1e-3 > df_map[gen]["dualBound"]) | \
        ((df_map[gen]["dualBound"] - 1e-3 > df_map[gen]["primalBound"]) & (df_map[gen]["dualBound"] / df_map[gen]["primalBound"] > 1 + 1e-3)) | \
        (df_map[gen]["primalBound"] > 1e20) | \
        (0 > df_map[gen]["vpcGenerationTime"]) | \
        (df_map[gen]["vpcGenerationTime"] - 1e-3 > df_map[gen]["rootDualBoundTime"]) | \
        (df_map[gen]["rootDualBoundTime"] - 1e-3 > df_map[gen]["terminationTime"]) | \
        (df_map[gen]["vpcGenerationTime"] - 1e-3 > df_map[gen]["bestSolutionTime"]) | \
        (df_map[gen]["bestSolutionTime"] - 1e-3 > df_map[gen]["terminationTime"])
    print(f"{gen}: {mask.sum() / len(df_map[gen])}")
    df_map[gen] = df_map[gen][~mask]

None: 0.0006634599436059048
New: 0.0009084027252081756
Farkas: 0.0008265601322496211
NoDisjunction: 0.0011052777010223818


In [61]:
# merge the different data frames into one
join_cols = ["instance", "perturbation", "degree", "terms", "instanceIndex", "seedIndex"]
df = df_map[generators[0]].merge(df_map[generators[1]], on=join_cols, suffixes=(f" {generators[0]}", None))
for g1, g2 in zip(generators[1:-1], generators[2:]):
    df = df.merge(df_map[g2], on=join_cols, suffixes=(f" {g1}", None if g2 != generators[-1] else f" {g2}"))
df.head()

,instanceIndex,seedIndex,vpcGenerator None,terms,lpBound None,disjunctiveDualBound None,lpBoundPostVpc None,rootDualBound None,dualBound None,primalBound None,...,termRemainsFeasibleBasisInfeasible NoDisjunction,cutsChangedCoefficients NoDisjunction,feasibleTermsPrunedByBound NoDisjunction,tighten_disjunction NoDisjunction,tighten_matrix_perturbation NoDisjunction,tighten_infeasible_to_feasible_term NoDisjunction,tighten_feasible_to_infeasible_basis NoDisjunction,rows NoDisjunction,cols NoDisjunction,density NoDisjunction
0,0,0,None,64,-120.000000,-120.000000,-120.000000,-120.000000,-120.000000,-120.000000,...,0,0,0,0,0,0,0,8357,10735,0.000534
1,1,0,None,64,-120.000000,-120.000000,-120.000000,-120.000000,-120.000000,-120.000000,...,0,0,0,0,1,1,1,8357,10735,0.000534
2,2,0,None,64,-120.500000,-120.500000,-120.500000,-120.500000,-120.500000,-120.500000,...,0,0,0,0,1,1,1,8357,10735,0.000534
3,0,0,None,64,-4632.298153,-4632.298153,-4632.298153,-4631.571278,-4607.140232,-4606.679610,...,0,0,0,0,0,0,0,46,29,0.976762
4,1,0,None,64,-4628.667162,-4628.667162,-4628.667162,-4627.808946,-4604.833773,-4604.373375,...,0,0,0,0,1,1,1,46,29,0.976762


In [62]:
# get proportion of tests run to completion
len(generators) * len(df) / count_instances

0.5380228136882129

In [63]:
# assign nan's to experiments that didn't need to run - matrix support for RHS or any support for objective 
if filter_redundant:
    target_cols = [c for c in df.columns if any(s in c for s in [" NoDisjunction", " All"])
                   and any(metric in c for metric in ["Bound", "Time", "nodes", "iterations"])]
    df.loc[df["perturbation"] == "objective", target_cols] = np.nan

In [64]:
def gap_closed(df, col):
    gap = abs(df[col] - df["lpBound None"]) / abs(df['primalBound None'] - df["lpBound None"])
    gap[(gap > 1) | (gap == np.nan)] = 1  # get corner cases
    return gap

# Function to map values based on a dictionary
def check_same_solution(row):
    # Create a tuple of the key based on the key_columns
    return same_solution[row["perturbation"], int(math.log2(row["degree"])), f'{row["instance"]}_{row["instanceIndex"]}']

In [65]:
# find the optimality gap closed by each generator
df["Disjunction (New)"] = gap_closed(df, "disjunctiveDualBound New")
df["Disjunction (Old)"] = gap_closed(df, "disjunctiveDualBound Farkas")
for g in generators:
    if g != "None":
        df[f"VPCs ({g})"] = gap_closed(df, f"lpBoundPostVpc {g}")        
    df[f"Root Cuts ({g})"] = gap_closed(df, f"rootDualBound {g}")

df["Root Optimality Gap Improvement"] = df["Root Cuts (Farkas)"] - df["Root Cuts (None)"] 
# df = df.dropna()

In [66]:
# find times without vpc generation
df["terminationTimeSansVpc None"] = df["terminationTime None"]
df["rootDualBoundTimeSansVpc None"] = df["rootDualBoundTime None"]
for gen in generators:
    if gen != "None":
        df[f"terminationTimeSansVpc {gen}"] = df[f"terminationTime {gen}"] - df[f"vpcGenerationTime {gen}"]
        df[f"rootDualBoundTimeSansVpc {gen}"] = df[f"rootDualBoundTime {gen}"] - df[f"vpcGenerationTime {gen}"]
    df[f"postRootTime {gen}"] = df[f"terminationTime {gen}"] - df[f"rootDualBoundTime {gen}"]
    if gen not in ["None", "New"]:
        df[f"terminationTimeImprovement {gen}"] = (df["terminationTime None"] - df[f"terminationTime {gen}"]) / df["terminationTime None"]
        df[f"terminationTimeSansVpcImprovement {gen}"] = (df["terminationTimeSansVpc None"] - df[f"terminationTimeSansVpc {gen}"]) / df["terminationTimeSansVpc None"]
        df[f"nodesImprovement {gen}"] = (df["nodes None"] - df[f"nodes {gen}"]) / df["nodes None"] 
        df[f"iterationsImprovement {gen}"] = (df["iterations None"] - df[f"iterations {gen}"]) / df["iterations None"] 
        df[f"terminationTimeRatio {gen}"] = df[f"terminationTime {gen}"] / df["terminationTime None"]
        df[f"terminationTimeSansVpcRatio {gen}"] = df[f"terminationTimeSansVpc {gen}"] / df["terminationTimeSansVpc None"]
        df[f"nodesRatio {gen}"] = df[f"nodes {gen}"] / df["nodes None"] 
        df[f"iterationsRatio {gen}"] = df[f"iterations {gen}"] / df["iterations None"]
        df[f"nodesImproves {gen}"] = df["nodes None"] > df[f"nodes {gen}"]
        df[f"terminationTimeImproves {gen}"] = df["terminationTime None"] > df[f"terminationTime {gen}"]
        df[f"terminationTimeSansVpcImproves {gen}"] = df["terminationTimeSansVpc None"] > df[f"terminationTimeSansVpc {gen}"]
        df[f"iterationsImproves {gen}"] = df["iterations None"] > df[f"iterations {gen}"]
        
# df[f'{metric}Win{gen}'] = df[[f'{metric} {gen2}' for gen2 in compare_gens]].mean(axis=1) - 3 * df[[f'{metric} {gen2}' for gen2 in compare_gens]].std(axis=1) > df[f'{metric} {gen}']
for metric in ["nodes", "terminationTime", "terminationTimeSansVpc", "iterations"]:
    
    # does the generator win against all others?
    individuals = ["None"] + test_generators
    for gen in individuals:
        df[f'{metric}Win{gen}VsAll'] = pd.concat([
            pd.Series(
                np.where(
                    df[f'{metric} {gen}'].isna(), False,
                    np.where(
                        df[f'{metric} {gen2}'].isna(), True,
                        df[f'{metric} {gen2}'] * (1 - win_threshold) > df[f'{metric} {gen}']
                    )
                ),
                index=df.index
            )
            for gen2 in generators if gen2 != gen
        ], axis=1).all(axis=1)
        
    # does the disjunctive generator win against None?
    for gen in disjunctive_generators: 
        df[f'{metric}Win{gen}'] = pd.Series(
                np.where(
                    df[f'{metric} {gen}'].isna(), False,
                    df[f'{metric} None'] * (1 - win_threshold) > df[f'{metric} {gen}']
                ), index=df.index
            )

    # does any disjunctive generator win against None?
    df[f'{metric}WinAny'] = pd.concat([
        pd.Series(
            np.where(
                df[f'{metric} {gen}'].isna(), False,
                df[f'{metric} None'] * (1 - win_threshold) > df[f'{metric} {gen}']
            ),
            index=df.index
        )
        for gen in disjunctive_generators
    ], axis=1).any(axis=1)
    
    # does any parametric generator win against None?
    df[f'{metric}WinParametric'] = pd.concat([
        pd.Series(
            np.where(
                df[f'{metric} {gen}'].isna(), False,
                df[f'{metric} None'] * (1 - win_threshold) > df[f'{metric} {gen}']
            ),
            index=df.index
        )
        for gen in parametric_generators
    ], axis=1).any(axis=1)
    
    # does any strengthened parametric generator win against None?
    df[f'{metric}WinStrengthened'] = pd.concat([
        df[f"{metric}Win{gen}"] for gen in test_generators
        ], axis=1).any(axis=1)

df["bracket"] = ["short" if t <= short else "medium" if t <= medium else "long" for t in df["terminationTime None"]]
df["sameSolution"] = df.apply(check_same_solution, axis=1)

/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_46591/2909675915.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{metric}Win{gen}'] = pd.Series(
/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_46591/2909675915.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{metric}WinAny'] = pd.concat([
/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_46591/2909675915.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

In [67]:
# get sensitivity stats as ratios
for gen_name in generators:
    if gen_name == "None":
        continue
    df[f"infeasibleTermsRatio {gen_name}"] = df[f"infeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]
    df[f"infeasibleToFeasibleTermsRatio {gen_name}"] = df[f"infeasibleToFeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]
    df[f"zeroInfeasibleToFeasibleTerms {gen_name}"] = df[f"infeasibleToFeasibleTerms {gen_name}"] == 0
    df[f"feasibleToInfeasibleTermsRatio {gen_name}"] = df[f"feasibleToInfeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]

/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_46591/393337708.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"infeasibleTermsRatio {gen_name}"] = df[f"infeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]
/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_46591/393337708.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"infeasibleToFeasibleTermsRatio {gen_name}"] = df[f"infeasibleToFeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]
/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn

In [68]:
def optimality_gap(df, generator=None):
    if generator:
        return abs(df[f"primalBound {generator}"] - df[f"dualBound {generator}"]) / \
            abs(df[f"primalBound {generator}"])
    else:
        return abs(df[f"primalBound"] - df[f"dualBound"]) / abs(df[f"primalBound"])

In [69]:
# aleks filters
# df = df.loc[df["terms"] == df["actualTerms Farkas"]]
# df = df.loc[df["zeroInfeasibleToFeasibleTerms Farkas"]]

In [70]:
activation_metrics = ["infeasibleToFeasibleTerms", "termRemainsFeasibleBasisInfeasible", "cutsChangedCoefficients", "feasibleTermsPrunedByBound"]
for g in test_generators:
    df[f"active {g}"] = pd.concat([df[f"{m} {g}"] > 0 for m in activation_metrics], axis=1).any(axis=1)
print(df.loc[df[[f"active {g}" for g in test_generators]].all(axis=1)].shape[0], df.shape[0])
df = df.loc[df[[f"active {g}" for g in test_generators]].all(axis=1)]

1825 6509


/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_46591/406862815.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"active {g}"] = pd.concat([df[f"{m} {g}"] > 0 for m in activation_metrics], axis=1).any(axis=1)


In [71]:
# filter experiments where strengthening helps
# initial parametric disjunctive cut generation happens for terms with bases both initially and previously feasible
# for matrix perturbations, this can shift the direction of the cut relative to default parametric disjunctive cut generation
# which creates the edge case that the cut (although supporting) may not strengthen the root node as much
for g in test_generators:
    df[f"bad ({g})"] = df[f"VPCs ({g})"] < df["VPCs (Farkas)"] - 1e-6
print(df.loc[df[[f"bad ({g})" for g in test_generators]].any(axis=1)].shape[0], df.shape[0])
df = df.loc[~df[[f"bad ({g})" for g in test_generators]].any(axis=1)]

94 1825


/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_46591/1536581963.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"bad ({g})"] = df[f"VPCs ({g})"] < df["VPCs (Farkas)"] - 1e-6


In [72]:
# set aside core columns and filter for all subsequent dataframes
group_cols = ["instance", "perturbation", "bracket", "degree", "terms"]
id_cols = ["instanceIndex"]

# keep the instance, perturbation, instanceIndex triples that exist for all combinations of degree and terms
# where VPC did not find the optimal solution
full_df = df.loc[df["Disjunction (New)"] < .9999]
triples = (full_df.groupby(
        ["instance", "perturbation", "instanceIndex"]
    ).size().reset_index().rename(columns={0: "count"}))
triples.head()

,instance,perturbation,instanceIndex,count
0,10teams,matrix,1,2
1,10teams,matrix,2,2
2,10teams,matrix,3,1
3,10teams,matrix,4,2
4,22433,matrix,1,1


In [73]:
# uncomment to filter for only the triples that exist for all combinations of degree and terms (and seed index)
triples = triples[(triples["count"] == len(degrees) * len(term_list) * len(seed_idxs))]
full_df = full_df.merge(triples, on=["instance", "perturbation", "instanceIndex"])
full_df.to_csv(os.path.join(out_fldr, "cleaned_combined_complete.csv"), index=False, mode="w")

## Check Root Node Stats

In [74]:
def interleave(list_of_lists):
    return [item for sublist in zip(*list_of_lists) for item in sublist]

In [75]:
# additional filtering for dataframe on bounds
fields = ["Disjunction (New)", "Disjunction (Old)"] + [f"VPCs ({gen_name})" for gen_name in generators if gen_name != "None"] + \
    interleave([[f"Root Cuts ({gen_name})", f"terminationTime {gen_name}", f"nodes {gen_name}",
                 f"iterations {gen_name}", f"terminationTimeSansVpc {gen_name}", f"vpcGenerationTime {gen_name}", 
                 f"rootDualBoundTime {gen_name}"]
                for gen_name in generators]) + \
    interleave([[f"infeasibleTermsRatio {gen_name}", f"infeasibleToFeasibleTermsRatio {gen_name}",
                 f"zeroInfeasibleToFeasibleTerms {gen_name}", f"feasibleToInfeasibleTermsRatio {gen_name}"]
                for gen_name in generators if gen_name != "None"])

# now reduce bound_df to just the perturbed instances - make > -1 to include base instance
bound_df = full_df.loc[full_df["instanceIndex"] > 0, group_cols + id_cols + fields]  #  & (full_df["Disjunction (Old)"] > .1)

In [76]:
def geometric_mean(series, offset=1e-6):
    adjusted_series = series + offset  # Add a small offset to avoid zeros
    return np.exp(np.log(adjusted_series).mean())

# paper currently uses mean, but we can switch to geometric mean if we want
aggregations = {f: "mean" if not "Time" in f else geometric_mean for f in fields}  # geometric_mean if f not in ["sameSolution"] else 
aggregations["instance"] = "nunique"
aggregations["instanceIndex"] = "count"

In [77]:
# now break it down by type of perturbation
out = bound_df.groupby(["degree", "terms", "perturbation"]).agg(aggregations).reset_index()
out.to_csv(os.path.join(out_fldr, "bound_table_by_perturbation.csv"), index=False, mode="w")
out

,degree,terms,perturbation,Disjunction (New),Disjunction (Old),VPCs (New),VPCs (Farkas),VPCs (NoDisjunction),Root Cuts (None),Root Cuts (New),...,infeasibleToFeasibleTermsRatio Farkas,infeasibleToFeasibleTermsRatio NoDisjunction,zeroInfeasibleToFeasibleTerms New,zeroInfeasibleToFeasibleTerms Farkas,zeroInfeasibleToFeasibleTerms NoDisjunction,feasibleToInfeasibleTermsRatio New,feasibleToInfeasibleTermsRatio Farkas,feasibleToInfeasibleTermsRatio NoDisjunction,instance,instanceIndex
0,0.5,4,matrix,0.059627,0.026738,0.033847,0.018968,0.018971,0.733448,0.732555,...,0.000000,0.000000,1.0,1.000000,1.000000,0.0,0.033019,0.033019,45,106
1,0.5,4,rhs,0.085832,0.036917,0.033215,0.022349,0.022350,0.502030,0.517601,...,0.000000,0.000000,1.0,1.000000,1.000000,0.0,0.010204,0.010204,24,49
2,0.5,64,matrix,0.126071,0.078844,0.059426,0.033124,0.035107,0.734192,0.735780,...,0.015051,0.015051,1.0,0.877358,0.877358,0.0,0.020522,0.020522,45,106
3,0.5,64,rhs,0.176573,0.099886,0.111565,0.052122,0.064699,0.497214,0.534731,...,0.016331,0.016331,1.0,0.816327,0.816327,0.0,0.059135,0.059135,24,49
4,2.0,4,matrix,0.083318,0.025299,0.043049,0.010985,0.010998,0.803965,0.802421,...,0.004717,0.004717,1.0,0.990566,0.990566,0.0,0.084906,0.084906,45,106
5,2.0,4,rhs,0.186308,0.030026,0.031413,0.004750,0.004750,0.537698,0.528024,...,0.000000,0.000000,1.0,1.000000,1.000000,0.0,0.076531,0.076531,24,49
6,2.0,64,matrix,0.162636,0.057531,0.074326,0.004402,0.011787,0.801236,0.806068,...,0.074696,0.074696,1.0,0.547170,0.547170,0.0,0.080980,0.080980,45,106
7,2.0,64,rhs,0.273820,0.073100,0.091325,0.010757,0.011576,0.539096,0.563493,...,0.028996,0.028996,1.0,0.734694,0.734694,0.0,0.196095,0.196095,24,49


In [78]:
def make_pareto_frontier(bound_df, save_fig=True):
    # Identify relevant fields
    strength_fields = [f for f in fields if "Root Cuts" in f]
    time_fields = [f for f in fields if "vpcGenerationTime" in f]

    # Compute means
    strength_df = bound_df[strength_fields].mean().reset_index()
    strength_df.columns = ["key", "value"]
    strength_df['category'] = strength_df['key'].str.extract(r'\((.*?)\)')

    time_df = bound_df[time_fields].apply(geometric_mean).reset_index()
    time_df.columns = ["key", "value"]
    time_df['category'] = time_df['key'].str.extract(r'(None|Farkas|Old|New|All|NoDisjunction|Disjunction|Matrix|Term|Basis)')

    # Merge on category
    merged_df = pd.merge(strength_df, time_df, on='category', suffixes=('_strength', '_time'))
    merged_df['category'] = merged_df['category'].replace(cat_map)

    # Plotting
    plt.figure(figsize=(6, 5))
    categories = merged_df['category'].unique()
    cmap = plt.get_cmap('tab10')

    for i, category in enumerate(categories):
        sub_df = merged_df[merged_df['category'] == category]
        plt.scatter(
            sub_df['value_time'],
            sub_df['value_strength'],
            label=category,
            color=cmap(i % 10),
            s=25
        )

    plt.ylabel("Average Root Nodes\nOptimality Gap Closed")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1.0, 1))
    plt.xlabel("Average Time (s) to Process VPCs")
    plt.title("Root Nodes Optimality Gap Closed vs. Processing Time")
    plt.grid(True)
    plt.legend(title="Generator", loc="best", fontsize=12, title_fontsize=14)
    plt.tight_layout()

    if save_fig:
        plt.savefig(os.path.join(out_fldr, "strength_vs_time.png"), dpi=1200)

    print(merged_df.sort_values("value_strength", ascending=True)[["key_strength", "value_strength", "value_time"]])
    plt.show()


In [79]:
# again nearly pareto optimal - time and strength both ordered in terms of doing more "work". Makes sense for matrix case compared to 
# make_pareto_frontier(bound_df)

## Root Stats

In [80]:
# example table for VPC strength
# it isn't surprising to see improvements in 4 terms even though they don't improve at the root node because these cuts can be focused deeper in the tree
gens = ["New"] + test_generators + ["Farkas"]
out[["degree", "terms", "perturbation"] + [c for c in out.columns if "Disjunction (" in c] + [f"VPCs ({gen})" for gen in gens]].round(4)

,degree,terms,perturbation,Disjunction (New),Disjunction (Old),VPCs (New),VPCs (NoDisjunction),VPCs (Farkas)
0,0.5,4,matrix,0.0596,0.0267,0.0338,0.0190,0.0190
1,0.5,4,rhs,0.0858,0.0369,0.0332,0.0223,0.0223
2,0.5,64,matrix,0.1261,0.0788,0.0594,0.0351,0.0331
3,0.5,64,rhs,0.1766,0.0999,0.1116,0.0647,0.0521
4,2.0,4,matrix,0.0833,0.0253,0.0430,0.0110,0.0110
5,2.0,4,rhs,0.1863,0.0300,0.0314,0.0048,0.0048
6,2.0,64,matrix,0.1626,0.0575,0.0743,0.0118,0.0044
7,2.0,64,rhs,0.2738,0.0731,0.0913,0.0116,0.0108


In [81]:
# example table for root cut strength
gens = ["New"] + test_generators + ["Farkas", "None"]
out[["degree", "terms", "perturbation"] + [f"Root Cuts ({gen})" for gen in gens]].round(4)

,degree,terms,perturbation,Root Cuts (New),Root Cuts (NoDisjunction),Root Cuts (Farkas),Root Cuts (None)
0,0.5,4,matrix,0.7326,0.7325,0.7362,0.7334
1,0.5,4,rhs,0.5176,0.4847,0.4965,0.5020
2,0.5,64,matrix,0.7358,0.7359,0.7340,0.7342
3,0.5,64,rhs,0.5347,0.5302,0.5269,0.4972
4,2.0,4,matrix,0.8024,0.8039,0.8041,0.8040
5,2.0,4,rhs,0.5280,0.5245,0.5289,0.5377
6,2.0,64,matrix,0.8061,0.8024,0.7939,0.8012
7,2.0,64,rhs,0.5635,0.5347,0.5317,0.5391


In [82]:
# example table for root cut generation time
out[["degree", "terms", "perturbation"] + [f"rootDualBoundTime {gen}" for gen in gens]].round(3)

,degree,terms,perturbation,rootDualBoundTime New,rootDualBoundTime NoDisjunction,rootDualBoundTime Farkas,rootDualBoundTime None
0,0.5,4,matrix,1.394,0.480,0.435,0.314
1,0.5,4,rhs,0.690,0.297,0.300,0.275
2,0.5,64,matrix,11.878,2.135,1.061,0.336
3,0.5,64,rhs,6.428,0.744,0.549,0.313
4,2.0,4,matrix,2.450,0.377,0.373,0.296
5,2.0,4,rhs,0.794,0.248,0.234,0.239
6,2.0,64,matrix,9.564,1.704,1.211,0.288
7,2.0,64,rhs,4.358,0.546,0.515,0.234


In [83]:
effect_df = full_df[full_df["instanceIndex"] > 0]

target_cols = ["infeasibleToFeasibleTerms", "termRemainsFeasibleBasisInfeasible", "cutsChangedCoefficients",
               "numCuts", "feasibleTermsPrunedByBound"]
end_cols = ["infeasibleToFeasibleTerms", "termRemainsFeasibleBasisInfeasible", "cutsChangedCoefficientsRatio", "feasibleTermsPrunedByBound"]

gb = effect_df.groupby(["perturbation", "degree", "terms"]).agg(
    {f"{c} {gen}": "mean" for gen in generators for c in target_cols if gen in ["Disjunction", "NoDisjunction", "All"]} | {"instance": "nunique", "instanceIndex": "count"}    
)

for gen in test_generators:
    gb[f"cutsChangedCoefficientsRatio {gen}"] = gb[f"cutsChangedCoefficients {gen}"] / gb[f"numCuts {gen}"]
    
gb = gb[[f"{c} {gen}" for gen in generators for c in end_cols if gen in ["Disjunction", "NoDisjunction", "All"]] + ["instance", "instanceIndex"]]

In [84]:
# prune by bound higher for .5 degree makes sense given previous solutions more likely to be feasible
gb.loc[["matrix", "rhs"], [c for c in gb.columns if " NoDisjunction" in c or "instance" in c]].round(4)

infeasibleToFeasibleTerms NoDisjunction  \
perturbation degree terms                                            
matrix       0.5    4                                       0.0000   
                    64                                      1.1132   
             2.0    4                                       0.0189   
                    64                                      7.7075   
rhs          0.5    4                                       0.0000   
                    64                                      1.1224   
             2.0    4                                       0.0000   
                    64                                      2.1429   

                           termRemainsFeasibleBasisInfeasible NoDisjunction  \
perturbation degree terms                                                     
matrix       0.5    4                                                2.2736   
                    64                                              20.8962   
             2.0    4                                                2.9340   
                    64                                              25.0094   
rhs          0.5    4                                                3.6735   
                    64                                              40.5714   
             2.0    4                                                3.6735   
                    64                                              36.9796   

                           cutsChangedCoefficientsRatio NoDisjunction  \
perturbation degree terms                                               
matrix       0.5    4                                          0.9271   
                    64                                         0.9790   
             2.0    4                                          0.9964   
                    64                                         0.9995   
rhs          0.5    4                                          0.0000   
                    64                                         0.0000   
             2.0    4                                          0.0000   
                    64                                         0.0000   

                           feasibleTermsPrunedByBound NoDisjunction  instance  \
perturbation degree terms                                                       
matrix       0.5    4                                           0.0        45   
                    64                                          0.0        45   
             2.0    4                                           0.0        45   
                    64                                          0.0        45   
rhs          0.5    4                                           0.0        24   
                    64                                          0.0        24   
             2.0    4                                           0.0        24   
                    64                                          0.0        24   

                           instanceIndex  
perturbation degree terms                 
matrix       0.5    4                106  
                    64               106  
             2.0    4                106  
                    64               106  
rhs          0.5    4                 49  
                    64                49  
             2.0    4                 49  
                    64                49

In [85]:
# gb.loc["objective", [c for c in gb.columns if " Disjunction" in c or "instance" in c]]

In [86]:
# gb.loc["rhs", [c for c in gb.columns if " NoDisjunction" in c  or "instance" in c]].round(4)

## Check Termination Stats

In [87]:
# only check perturbed instances that solve to optimality and VPC didn't find optimal solution
mask = (
    (df["Disjunction (New)"] < 0.9999) & (df["instanceIndex"] > 0)
    & np.logical_and.reduce([(optimality_gap(df, gen) <= 1e-4) | (pd.isnull(optimality_gap(df, gen))) for gen in generators])
    & (df["terminationTime None"] > min_termination_time) & (df["perturbation"] != "objective")
)
# if remove_status_changes:
#     mask = mask & (df["infeasibleToFeasibleTermsRatio Farkas"] == 0) & (df["feasibleToInfeasibleTermsRatio Farkas"] == 0)

gap_df = df.loc[mask]

# # reduce to only ones where all terms and degrees are present
# triples = (gap_df.groupby(
#         ["instance", "perturbation", "instanceIndex"]
#     ).size().reset_index().rename(columns={0: "count"}))
# triples = triples[triples["count"] == len(degrees) * len(term_list) * len(seed_idxs)]
# gap_df = gap_df.merge(triples, on=["instance", "perturbation", "instanceIndex"])

In [88]:
def plot_distributions(histogram_df, feature, bins=100, xlim=(-2, 1), ylim=(0, 1), perturbation=None, exclude_perturbation=False, title_x=.525, relative=True):
    """
    Generate a grid of cumulative distribution functions (CDFs) for a given feature,
    one for each combination of terms and degrees.
    """

    unique_degrees = histogram_df['degree'].sort_values(ascending=False).unique()
    unique_terms = histogram_df['terms'].sort_values().unique()

    fig, axes = plt.subplots(len(unique_degrees), len(unique_terms),
                             figsize=(4 * len(unique_terms), 4 * len(unique_degrees)))
    
    compare_gens = [gen for gen in generators if gen != "None" and gen != "New"]

    for i, degree in enumerate(unique_degrees):
        for j, terms in enumerate(unique_terms):
            ax = axes[i, j] if len(unique_degrees) > 1 and len(unique_terms) > 1 else axes[i] if len(unique_degrees) > 1 \
                else axes[j] if len(unique_terms) > 1 else axes

            subset_df = histogram_df[(histogram_df['degree'] == degree) & (histogram_df['terms'] == terms)]

            if perturbation is not None:
                subset_df = subset_df[subset_df["perturbation"] == perturbation] if not exclude_perturbation \
                    else subset_df[subset_df["perturbation"] != perturbation]

            # Compute relative improvements dynamically from generators
            relative_improvements = {gen: subset_df[f"{feature} {gen}"] for gen in compare_gens} if not relative else \
                {gen: -(subset_df[f"{feature} None"] - subset_df[f"{feature} {gen}"]) / subset_df[f"{feature} None"] for gen in compare_gens}
            
            # filter out nan's and drop groups that are emptied - e.g. any supporting for objective perturbations or matrix supporting for rhs
            relative_improvements = {gen: ri[ri.notna()] for gen, ri in relative_improvements.items() if ri.notna().sum() > 1}

            x = np.linspace(xlim[0], xlim[1], bins)

            # Compute CDFs
            cdfs = {
                gen: np.array([(ri <= val).sum() / len(ri) for val in x])
                for gen, ri in relative_improvements.items()
            }

            # Plot each generator's CDF with distinct color
            cmap = plt.get_cmap("tab10")
            for gen in cdfs:
                ax.plot(x, cdfs[gen], label=cat_map.get(gen, gen), color=cmap(generators.index(gen)))

            # Optionally fill region where first generator dominates all others
            primary = "Farkas"
            others = [g for g in cdfs if g != "Farkas"]
            fill_region = [max(cdfs[gen][k] for gen in others) > cdfs[primary][k] for k in range(len(x))]
            ax.fill_between(x, cdfs[primary], np.maximum.reduce([cdfs[gen] for gen in others]),
                            where=fill_region,
                            facecolor='yellow', alpha=0.7,
                            label=f'Improvement over\n{cat_map.get(primary, primary)}')

            ax.set_xlim(xlim)
            ax.set_ylim(ylim)
            # if not relative:
            #     # set x-axis to log scale if not relative
            #     ax.set_xscale('log')
            ax.set_title(f"{degree} Degree{'s' if degree > 1 else ''}, {terms} Terms")
            if j == 0:
                ax.set_ylabel("Probability")
            if i == len(unique_degrees) - 1:
                ax.set_xlabel("Relative Change" if relative else "Solve Time (s)")

    # Shared legend
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(title_x, -0.1), ncol=3)

    fig.suptitle(f"CDF of {'Relative Change in ' if relative else ''}Solve {label[feature]}"
                 f"{' for ' + (perturbation.title() if perturbation != 'rhs' else perturbation.upper()) + ' Perturbations' if perturbation else ''}", x=title_x)
    plt.tight_layout()

    plt.savefig(os.path.join(out_fldr, f"cdf_{feature}{'_' + perturbation if perturbation else ''}{'_relative_improvements' if relative else ''}.png"), dpi=1200, bbox_inches='tight')
    plt.show()

In [89]:
# expect that fewer terms better given last paper shows generation time paramount
# time outweighs strength, so although relatively more bound closed at 2 degrees, more terms need tightening, which is very noticeable for supporting methods 
# then makes sense that for few terms and degrees that supporting methods do better but more more terms and degrees prune is better since its quick
# plot_distributions(gap_df, "terminationTime", xlim=(100, 3600), ylim=(.6, 1), bins=350, relative=False)
# plot_distributions(gap_df, "terminationTime", xlim=(-1, 0), ylim=(0, .5), bins=1000, relative=True)

In [90]:
# support methods outperform default parameterization more often for fewer terms than they do for more, which makes sense since generation time penalized less

# plot_distributions(gap_df, "terminationTime", xlim=(100, 3600), ylim=(.6, 1), bins=350, relative=False, perturbation="matrix")
# plot_distributions(gap_df, "terminationTime", xlim=(-1, 0), ylim=(0, .5), bins=1000, relative=True, perturbation="matrix")

In [91]:
# more terms -> more opportunity for incumbent to prune (thus more effective)
# fewer degrees -> no relationship in table for frequency, but .5, 64 did have most opportunity, so it makes sense it sees most improvement 
# wouldn't expect such a separation since the methods are similarly fast and pruning only makes it stronger
# plot_distributions(gap_df, "terminationTime", xlim=(100, 3600), ylim=(.6, 1), bins=350, relative=False, perturbation="objective")
# plot_distributions(gap_df, "terminationTime", xlim=(-1, 0), ylim=(0, .5), bins=1000, relative=True, perturbation="objective")

In [92]:
# not as surprising that we do better for more degrees given more opportunity for strengthening with support methods while root cut generation time does not vary much
# makes sense then that support methods do better than pruning alone for more degrees
# for fewer degrees, makes sense that pruning is better since we're still doing roughly the same work to generating supporting cuts as with more degrees but have less opportunity
# would have expected prune and support to do better than support since its as strong but quicker
# plot_distributions(gap_df, "terminationTime", xlim=(100, 3600), ylim=(.6, 1), bins=350, relative=False, perturbation="rhs")
# plot_distributions(gap_df, "terminationTime", xlim=(-1, 0), ylim=(0, .5), bins=1000, relative=True, perturbation="rhs")

In [93]:
def rename_cols(cols, feature):
    new_cols = []
    for col in cols:
        if f"{feature}Win" in col:
            col = col.replace(f"{feature}Win", f"{label[feature]} Win % ")
        elif "instanceIndex" in col:
            col = col.replace("instanceIndex", "Test Instances")
        elif "instance" in col:
            col = col.replace("instance", "Base Instances")
        new_cols.append(col)
    return new_cols

In [113]:
def get_wins(feature, grouping):
    """
    
    :param feature: "nodes", "terminationTime", "terminationTimeSansVpc"
    :param grouping: "perturbation", "terms", "bracket", "degree" 
    :return: 
    """

    # get the win percentages for each feature on average and broken down by grouping type
    features = [feature]
    keys = [] # removed perturbation since patterns did not exist - finding them means finding triples which just makes the test set too small
    wins = {}
    
    for feature in features:
        
        # define aggregating operations
        aggregations = {f"{feature}Win{gen}": "mean" for gen in ["NoneVsAll", "NoDisjunctionVsAll", "New"] + test_generators + ["Farkas", "Parametric", "Any"]}  # "Strengthened", 
        if feature == features[-1]:
            aggregations = aggregations | {"instance": "nunique", "instanceIndex": "count"}
        
        # find the average wins for the feature grouped by degree and terms
        feature_wins = gap_df[gap_df["perturbation"] != "bound"].groupby(keys + [grouping]).agg(aggregations)
        
        # clean up formatting
        feature_wins.columns = rename_cols(feature_wins.columns, feature)
        win_cols = [c for c in feature_wins.columns if "Win" in c]
        feature_wins[win_cols] = feature_wins[win_cols].applymap(lambda x: round(x * 100, 2))
        instance_cols = [c for c in feature_wins.columns if "instance" in c]
        feature_wins[instance_cols] = feature_wins[instance_cols].applymap(lambda x: int(x))
        
        # save the df
        wins[feature] = feature_wins
        
    # bring them all together now
    all_wins = pd.concat(wins.values(), axis=1).sort_values(keys + [grouping], ascending=[c != "bracket" for c in keys + [grouping]]).reset_index()
    all_wins.to_csv(os.path.join(out_fldr, f"branch_and_bound_wins_{feature}_{grouping}.csv"), index=False, mode="w")
    all_wins
    return all_wins

In [114]:
# general trends
# some form of disjunctive cuts leads to a significant improvement in nodes and time, which makes sense given the depth of the cuts
# disjunctive methods are all somewhat orthogonal to each other - motivates figuring out when to use when
# this includes strengthening
# to get a decent sample size, the same instances aren't included in both subsets here, so that can also throw things off
# todo it's fine that we don't see improvements in 4 terms at root. Disjunctive cuts are still known for their strength, which often comes down to their refining deeper in the tree (aleks) and we'll see benefit of them in later results


# rhs had better root node strength than matrix on average for parametric disjunctive cuts -> it does better
# in both nodes and time, farkas degrades more than strengthening does
# strengthening reduces nodes compared to Farkas
# would have expected more for matrix perturbations given more reparameterization, but prachi shows even if they are tighter that doesn't mean smaller trees
get_wins("nodes", "perturbation")

,perturbation,Nodes Processed Win % NoneVsAll,Nodes Processed Win % NoDisjunctionVsAll,Nodes Processed Win % New,Nodes Processed Win % NoDisjunction,Nodes Processed Win % Farkas,Nodes Processed Win % Parametric,Nodes Processed Win % Any,Base Instances,Test Instances
0,matrix,13.91,15.0,35.87,38.91,36.3,52.39,60.0,89,460
1,rhs,13.20,14.8,34.00,41.20,38.0,53.60,59.6,59,250


In [115]:
# in both nodes and time, farkas degrades more than strengthening does
# ratio of strengthening wins to farkas wins increases for matrix but reduces for rhs regarding time -> follows aleks pattern that dense cuts slow down LP solves thus run time
# higher valued new for nodes leads to lower valued new for time for same reason
# ratio of parametric wins to any wins increases from nodes to time for same reason
get_wins("terminationTime", "perturbation")

,perturbation,Time Win % NoneVsAll,Time Win % NoDisjunctionVsAll,Time Win % New,Time Win % NoDisjunction,Time Win % Farkas,Time Win % Parametric,Time Win % Any,Base Instances,Test Instances
0,matrix,24.78,22.39,25.43,40.0,36.52,54.57,59.78,89,460
1,rhs,18.40,18.40,30.00,39.2,42.80,56.00,61.60,59,250


In [116]:
# more degree, more impact strengthening has over farkas, this supporting parametric cuts improve relative to farkas
# farkas loses more often for more degrees in line with previous paper
# parameterization wins less often for more degrees (even if cuts tight, further from LP relaxation solution)
# for same reason, using new becomes more advantageous for more degrees
# would have expected supporting parametric cuts to degrade with more degrees, albeit less slowly than farkas, but it doesn't, which is reasonable here since the disjunctions for 2 degrees appear to be stronger than for .5
get_wins("nodes", "degree")

,degree,Nodes Processed Win % NoneVsAll,Nodes Processed Win % NoDisjunctionVsAll,Nodes Processed Win % New,Nodes Processed Win % NoDisjunction,Nodes Processed Win % Farkas,Nodes Processed Win % Parametric,Nodes Processed Win % Any,Base Instances,Test Instances
0,0.5,14.88,13.41,33.9,38.05,38.54,53.17,59.27,77,410
1,2.0,12.00,17.00,37.0,42.00,34.67,52.33,60.67,64,300


In [117]:
# we don't see a clear pattern here, but again that's not all that surprising given the unknown nature of when disjunctive cuts improve both nodes and run time (again citing aleks' paper)
get_wins("terminationTime", "degree")

,degree,Time Win % NoneVsAll,Time Win % NoDisjunctionVsAll,Time Win % New,Time Win % NoDisjunction,Time Win % Farkas,Time Win % Parametric,Time Win % Any,Base Instances,Test Instances
0,0.5,23.66,20.73,26.34,40.24,37.8,55.37,61.22,77,410
1,2.0,21.00,21.33,28.00,39.00,40.0,54.67,59.33,64,300


In [118]:
# strengthening's advantage over default parameterization grows for more terms, expected since more opportunity to strengthen
# would have expected more terms and stronger cuts to lead to fewer nodes, but again prachi's paper says otherwise
get_wins("nodes", "terms")

,terms,Nodes Processed Win % NoneVsAll,Nodes Processed Win % NoDisjunctionVsAll,Nodes Processed Win % New,Nodes Processed Win % NoDisjunction,Nodes Processed Win % Farkas,Nodes Processed Win % Parametric,Nodes Processed Win % Any,Base Instances,Test Instances
0,4,13.80,15.01,35.59,39.71,37.77,53.75,59.81,96,413
1,64,13.47,14.81,34.68,39.73,35.69,51.52,59.93,70,297


In [119]:
# disjunctive methods get less effective with more terms because cut generation time grows (last paper shows this hurts run time for parametric methods, aleks shows this for his own)
# knowing this relationship, makes sense we see significant separation between new and parametric methods for run time
# also makes sence farkas doesn't degrade as quickly and that default improves
get_wins("terminationTime", "terms")

,terms,Time Win % NoneVsAll,Time Win % NoDisjunctionVsAll,Time Win % New,Time Win % NoDisjunction,Time Win % Farkas,Time Win % Parametric,Time Win % Any,Base Instances,Test Instances
0,4,17.92,22.76,29.78,42.62,39.47,58.35,64.65,96,413
1,64,28.96,18.52,23.23,35.69,37.71,50.51,54.55,70,297


In [120]:
# longer runs are more advantageous for strengthening and new. those two make stronger cuts and so any subtree near the root that they prune has more impact the bigger the tree gets
# strengthening reinforces the pattern of more run time, more impact disjunctive cuts have that aleks noticed for new
# makes sense we see it for strengthen and new but not for farkas since the former support the disjunctive hull
# and therefore are more likely for longer runs to prune more of the tree
get_wins("nodes", "bracket")

,bracket,Nodes Processed Win % NoneVsAll,Nodes Processed Win % NoDisjunctionVsAll,Nodes Processed Win % New,Nodes Processed Win % NoDisjunction,Nodes Processed Win % Farkas,Nodes Processed Win % Parametric,Nodes Processed Win % Any,Base Instances,Test Instances
0,short,14.43,15.12,32.99,38.83,39.18,53.95,60.14,74,291
1,medium,14.71,15.07,35.66,39.34,33.82,51.47,59.56,60,272
2,long,10.20,14.29,38.78,42.18,38.10,53.06,59.86,36,147


In [121]:
# short runs follow expected behavior where run time wins are less likely than node wins because of cut density slowing down LP solves
# what's interesting is that we have the opposite behavior for long runs, where somehow run time wins are more likely than node wins
# possibilities could include that the strength of disjunctive hull supporting cuts allows for more cut generation to be skipped, or
# the tree could be shallower, leading to better parallelization.
get_wins("terminationTime", "bracket")

,bracket,Time Win % NoneVsAll,Time Win % NoDisjunctionVsAll,Time Win % New,Time Win % NoDisjunction,Time Win % Farkas,Time Win % Parametric,Time Win % Any,Base Instances,Test Instances
0,short,32.65,20.27,14.09,36.43,35.40,51.20,53.95,74,291
1,medium,17.65,22.79,31.25,39.34,41.18,56.99,63.60,60,272
2,long,11.56,19.05,44.90,46.94,40.82,59.18,67.35,36,147


In [103]:
def aggregate_wins(gap_df, feature, grouping):

    def custom_key(col):
        # bump improvement columns second to win percentage columns
        offset = int("Improvement %" in col)
        # check groupings
        if 'matrix' in col or 'short' in col:
            return (2 + offset, col)
        elif 'objective' in col or 'medium' in col:
            return (4 + offset, col)
        elif 'rhs' in col or 'long' in col:
            return (6 + offset, col)    
        return (offset, col)
    
    # find the average wins for the feature grouped by degree, terms and grouping type
    win_aggregations = {f"{feature}Win{gen}": "mean" for gen in generators + ["Any"]}
    feature_wins = gap_df[gap_df["perturbation"] != "bound"].groupby(["degree", "terms", grouping]).agg(win_aggregations).reset_index().pivot(
        index=['degree', 'terms'], columns=grouping, values=[f"{feature}Win{gen}" for gen in generators + ["Any"]]
    )
    feature_wins.columns = rename_cols([' '.join(col).strip() for col in feature_wins.columns.values], feature)
    feature_wins = feature_wins[sorted(feature_wins.columns, key=custom_key)]
    feature_wins = feature_wins.applymap(lambda x: round(x * 100, 2))
    
    # get the counts for the feature grouped by degree, terms and grouping type
    count_aggregations = {"instance": "nunique", "instanceIndex": "count"}
    feature_counts = gap_df[gap_df["perturbation"] != "bound"].groupby(["degree", "terms", grouping]).agg(count_aggregations).reset_index().pivot(
        index=['degree', 'terms'], columns=grouping, values=["instance", "instanceIndex"]
    )
    feature_counts.columns = rename_cols([' '.join(col).strip() for col in feature_counts.columns.values], feature)
    feature_counts = feature_counts[sorted(feature_counts.columns, key=custom_key)]
    feature_counts = feature_counts.applymap(lambda x: int(x))
    
    # save wins, base instance counts, and test instance counts to csv
    feature_wins.reset_index().to_csv(os.path.join(out_fldr, f"branch_and_bound_wins_{feature}_{grouping}.csv"), index=False, mode="w")
    feature_counts.reset_index().to_csv(os.path.join(out_fldr, f"branch_and_bound_counts_{grouping}.csv"), index=False, mode="w")
    
    return feature_wins, feature_counts

In [104]:
# wins, counts = aggregate_wins(gap_df, "terminationTime", "bracket")
# wins

In [105]:
# wins, counts = aggregate_wins(gap_df, "terminationTime", "perturbation")
# wins

## High Performing Run Time Subset

In [106]:
# additional filtering for dataframe on run time
fields = [f"terminationTime {gen}" for gen in generators] + \
         [f"terminationTimeImprovement {gen}" for gen in generators if gen not in ["None", "New"]]

# create time dataframe
time_df = df.loc[mask, group_cols + id_cols + fields]

In [107]:
aggregations = {f"Average Time {gen}": (f"terminationTime {gen}", geometric_mean) for gen in generators} | \
    {f"Average Improvement {gen}": (f"terminationTimeImprovement {gen}", "mean") for gen in generators if gen not in ["None", "New"]} | \
    {"count": ("terminationTimeImprovement Farkas", "size")}

tmp = time_df.groupby(["instance", "perturbation", "degree", "terms"]).agg(**aggregations).reset_index()
tmp = tmp[(tmp["count"] > 1)]
tmp.to_csv(os.path.join(out_fldr, "high_perform_all.csv"), index=False, mode="w")
tmp.head()

,instance,perturbation,degree,terms,Average Time None,Average Time New,Average Time Farkas,Average Time NoDisjunction,Average Improvement Farkas,Average Improvement NoDisjunction,count
1,a1c1s1,rhs,0.5,64,65.925121,186.154677,44.346425,82.921954,0.268115,-0.370371,2
4,a2c1s1,matrix,0.5,4,1525.196523,1311.996049,2987.881641,1420.511332,-0.969961,0.067408,2
5,a2c1s1,matrix,0.5,64,1612.793921,1762.012670,1883.762620,1995.933467,-0.282755,-0.381003,4
6,a2c1s1,rhs,0.5,4,35.506289,43.720299,35.433677,26.455276,-0.037295,0.254900,2
7,a2c1s1,rhs,0.5,64,28.447190,38.998171,26.594088,24.438430,0.035426,0.105769,4


In [108]:
def make_improvement_table(tmp, generator):
    
    # columns we always choose
    key_cols = ["degree", "terms", "perturbation", "instance"]
    time_cols = [f"Average Time {g}" for g in ["None", "New", "Farkas"]]
    
    # subset the ones we want
    all_df = tmp[
        key_cols + time_cols + [f"Average Time {generator}", f"Average Improvement Farkas", f"Average Improvement {generator}", "count"]
    ].sort_values(f"Average Improvement {generator}", ascending=False)
    all_df = all_df[
        (all_df[f"Average Improvement {generator}"] > 0) & 
        (all_df["Average Time Farkas"] > 1.2 * all_df[f"Average Time {generator}"]) & 
        (all_df["Average Time New"] > 1.2 * all_df[f"Average Time {generator}"])
    ]
    best_df = all_df.loc[
        all_df.groupby(['degree', 'terms', 'perturbation'])[f'Average Improvement {generator}'].idxmax()
    ]  # .sort_values(f"Average Improvement {generator}", ascending=False).round(2)
    
    # save all the winners
    all_df.to_csv(os.path.join(out_fldr, f"high_perform_{generator.lower()}.csv"), index=False, mode="w")
    
    # return just the best
    return all_df, best_df

In [109]:
# generator = "Disjunction"
# all_df, best_df = make_improvement_table(tmp, generator)
# best_df[["degree", "terms", "perturbation", "instance", "Average Time None", "Average Time New", "Average Time Farkas", f"Average Time {generator}", "count"]].head(3)

In [110]:
generator = "NoDisjunction"
all_df, best_df = make_improvement_table(tmp, generator)
best_df[["degree", "terms", "perturbation", "instance", "Average Time None", "Average Time New", "Average Time Farkas", f"Average Time {generator}", "count"]]

,degree,terms,perturbation,instance,Average Time None,Average Time New,Average Time Farkas,Average Time NoDisjunction,count
45,0.5,4,matrix,danoint,1481.411435,1596.665227,1348.723483,657.528542,3
64,0.5,4,rhs,gen-ip002,1379.712370,1183.200028,1318.780735,695.452522,2
107,0.5,64,matrix,k16x240b,314.771144,243.415143,245.650637,173.607462,3
282,0.5,64,rhs,timtab1,243.951679,251.254951,218.962811,172.204685,4
279,2.0,4,matrix,timtab1,56.181840,55.777320,44.019632,28.522040,2
78,2.0,4,rhs,gen-ip036,295.698084,250.504192,265.263592,193.198810,4
69,2.0,64,matrix,gen-ip021,384.302044,439.966624,359.611187,204.901125,4
105,2.0,64,rhs,icir97_tension,1027.737175,1343.064551,1700.370790,953.794123,3


In [111]:
# generator = "All"
# all_df, best_df = make_improvement_table(tmp, generator)
# best_df[["degree", "terms", "perturbation", "instance", "Average Time None", "Average Time New", "Average Time Farkas", f"Average Time {generator}", "count"]].head(3)